In [1]:
import librosa
import librosa.display
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import glob
import shutil
import subprocess
import wave
from tensorflow.keras.utils import to_categorical
import keras
import re
from keras.applications.densenet import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, LSTM, GRU, SimpleRNN, Conv1DTranspose, Concatenate, Conv2DTranspose
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Task1_nonsegment

In [2]:
# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_nonsegment/Healthy'
# 읽어올 폴더 경로 설정(데이터 길이 구하기 위함)
data_folder_unhealthy = 'D:/Data/호흡데이터/Task1/Wav_nonsegment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list = []
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith('.wav'):
            file_list.append(os.path.join(root, file))
            
            
# 데이터 길이 맞추기위해 읽어올 파일 리스트 생성
file_list_unhealthy = []
for root, dirs, files_unhealthy in os.walk(data_folder_unhealthy):
    for file_unhealthy in files_unhealthy:
        if file.endswith('.wav'):
            file_list_unhealthy.append(os.path.join(root, file_unhealthy))
            

# 가장 짧은 파일의 길이 구하기
min_len = min([len(librosa.load(file_unhealthy)[0]) for file_unhealthy in file_list_unhealthy])

# 데이터프레임 생성
dfs = []
for file in file_list:
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 가장 짧은 길이에 맞게 데이터 자르기
    audio_data = audio_data[:min_len]
    # 시간 축 설정
    time_axis = librosa.times_like(audio_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(audio_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_healthy = pd.concat(dfs, axis=1, ignore_index=True)

In [3]:
df_healthy = df_healthy.transpose()
df_healthy

,0,1,2,3,4,5,6,7,8,9,...,173215,173216,173217,173218,173219,173220,173221,173222,173223,173224
0,0.002834,0.004234,0.003641,0.003971,0.003700,0.003809,0.003576,0.003688,0.003570,0.003566,...,0.008067,0.008140,0.008190,0.008175,0.008217,0.008314,0.008320,0.008422,0.008517,0.008680
1,-0.074944,-0.108564,-0.093635,-0.100122,-0.091679,-0.093961,-0.087420,-0.088095,-0.083677,-0.084414,...,0.001657,0.001447,0.000820,0.000003,-0.000793,-0.001621,-0.002303,-0.002842,-0.003067,-0.003418
2,0.031243,0.045788,0.040219,0.043209,0.039028,0.038991,0.034904,0.033426,0.029972,0.028974,...,-0.014146,-0.012259,-0.009135,-0.005213,-0.000988,0.002983,0.006162,0.008906,0.010976,0.012153
3,0.030055,0.043004,0.036623,0.038840,0.035038,0.035900,0.033086,0.033206,0.031031,0.030529,...,0.089253,0.088449,0.087667,0.086844,0.086066,0.085214,0.084552,0.083685,0.082866,0.082092
4,-0.058900,-0.084398,-0.071648,-0.076402,-0.070458,-0.074119,-0.071738,-0.075175,-0.074700,-0.077834,...,-0.206857,-0.213262,-0.218788,-0.223172,-0.226421,-0.228739,-0.230270,-0.231345,-0.232212,-0.233467
5,-0.007034,-0.010403,-0.009126,-0.009703,-0.009150,-0.009540,-0.009445,-0.009780,-0.009571,-0.009791,...,-0.000319,-0.000152,0.000040,0.000203,0.000336,0.000413,0.000378,0.000451,0.000369,0.000313
6,0.008411,0.012138,0.010478,0.011247,0.010533,0.011056,0.010151,0.009742,0.009111,0.009042,...,-0.035101,-0.035011,-0.035020,-0.035029,-0.035122,-0.034985,-0.034749,-0.033376,-0.033500,-0.033659
7,-0.035490,-0.052700,-0.047214,-0.051584,-0.049264,-0.052077,-0.050764,-0.052858,-0.052138,-0.053505,...,0.251070,0.251797,0.252594,0.253096,0.253664,0.254207,0.254856,0.255327,0.256061,0.256631
8,-0.007025,-0.010711,-0.009760,-0.011117,-0.010905,-0.011542,-0.011487,-0.012052,-0.011880,-0.012059,...,-0.043829,-0.043033,-0.042452,-0.042494,-0.042305,-0.042105,-0.041877,-0.041487,-0.040987,-0.040699
9,0.069494,0.102969,0.091331,0.099595,0.093863,0.098485,0.095500,0.098237,0.096112,0.098258,...,-0.010475,-0.009911,-0.009007,-0.008181,-0.007259,-0.006685,-0.005691,-0.004556,-0.003732,-0.003431


In [4]:
df_healthy.rename(columns={0 : 'label'}, inplace=True)
df_healthy['label'] = 0
df_healthy

,label,1,2,3,4,5,6,7,8,9,...,173215,173216,173217,173218,173219,173220,173221,173222,173223,173224
0,0,0.004234,0.003641,0.003971,0.003700,0.003809,0.003576,0.003688,0.003570,0.003566,...,0.008067,0.008140,0.008190,0.008175,0.008217,0.008314,0.008320,0.008422,0.008517,0.008680
1,0,-0.108564,-0.093635,-0.100122,-0.091679,-0.093961,-0.087420,-0.088095,-0.083677,-0.084414,...,0.001657,0.001447,0.000820,0.000003,-0.000793,-0.001621,-0.002303,-0.002842,-0.003067,-0.003418
2,0,0.045788,0.040219,0.043209,0.039028,0.038991,0.034904,0.033426,0.029972,0.028974,...,-0.014146,-0.012259,-0.009135,-0.005213,-0.000988,0.002983,0.006162,0.008906,0.010976,0.012153
3,0,0.043004,0.036623,0.038840,0.035038,0.035900,0.033086,0.033206,0.031031,0.030529,...,0.089253,0.088449,0.087667,0.086844,0.086066,0.085214,0.084552,0.083685,0.082866,0.082092
4,0,-0.084398,-0.071648,-0.076402,-0.070458,-0.074119,-0.071738,-0.075175,-0.074700,-0.077834,...,-0.206857,-0.213262,-0.218788,-0.223172,-0.226421,-0.228739,-0.230270,-0.231345,-0.232212,-0.233467
5,0,-0.010403,-0.009126,-0.009703,-0.009150,-0.009540,-0.009445,-0.009780,-0.009571,-0.009791,...,-0.000319,-0.000152,0.000040,0.000203,0.000336,0.000413,0.000378,0.000451,0.000369,0.000313
6,0,0.012138,0.010478,0.011247,0.010533,0.011056,0.010151,0.009742,0.009111,0.009042,...,-0.035101,-0.035011,-0.035020,-0.035029,-0.035122,-0.034985,-0.034749,-0.033376,-0.033500,-0.033659
7,0,-0.052700,-0.047214,-0.051584,-0.049264,-0.052077,-0.050764,-0.052858,-0.052138,-0.053505,...,0.251070,0.251797,0.252594,0.253096,0.253664,0.254207,0.254856,0.255327,0.256061,0.256631
8,0,-0.010711,-0.009760,-0.011117,-0.010905,-0.011542,-0.011487,-0.012052,-0.011880,-0.012059,...,-0.043829,-0.043033,-0.042452,-0.042494,-0.042305,-0.042105,-0.041877,-0.041487,-0.040987,-0.040699
9,0,0.102969,0.091331,0.099595,0.093863,0.098485,0.095500,0.098237,0.096112,0.098258,...,-0.010475,-0.009911,-0.009007,-0.008181,-0.007259,-0.006685,-0.005691,-0.004556,-0.003732,-0.003431


In [5]:
# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_nonsegment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list = []
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith('.wav'):
            file_list.append(os.path.join(root, file))

# 가장 짧은 파일의 길이 구하기
min_len = min([len(librosa.load(file)[0]) for file in file_list])

# 데이터프레임 생성
dfs = []
for file in file_list:
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 가장 짧은 길이에 맞게 데이터 자르기
    audio_data = audio_data[:min_len]
    # 시간 축 설정
    time_axis = librosa.times_like(audio_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(audio_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_unhealthy = pd.concat(dfs, axis=1, ignore_index=True)

In [6]:
df_unhealthy = df_unhealthy.transpose()
df_unhealthy

,0,1,2,3,4,5,6,7,8,9,...,173215,173216,173217,173218,173219,173220,173221,173222,173223,173224
0,0.069739,0.102660,0.091162,0.099346,0.093962,0.098952,0.095814,0.099173,0.097429,0.099446,...,0.032574,0.031380,0.030319,0.029251,0.028082,0.027002,0.025912,0.024975,0.023997,0.022895
1,-0.027108,-0.039935,-0.035308,-0.038430,-0.036246,-0.038055,-0.036828,-0.038055,-0.037291,-0.037937,...,0.014216,0.013896,0.013550,0.013139,0.012629,0.012183,0.011777,0.011292,0.010910,0.010378
2,-0.254565,-0.370539,-0.323283,-0.349609,-0.325975,-0.340078,-0.324729,-0.333104,-0.322754,-0.327030,...,-0.315960,-0.317101,-0.318015,-0.319889,-0.322268,-0.324422,-0.325804,-0.326805,-0.328887,-0.330444
3,0.000032,0.000006,-0.000037,-0.000097,-0.000167,-0.000241,-0.000306,-0.000349,-0.000356,-0.000316,...,-0.057230,-0.058265,-0.059157,-0.059866,-0.060358,-0.060599,-0.060566,-0.060245,-0.059636,-0.058751
4,0.000793,0.000900,0.000911,0.000785,0.000488,-0.000004,-0.000697,-0.001581,-0.002624,-0.003781,...,0.404312,0.405778,0.406606,0.406793,0.406343,0.405275,0.403615,0.401401,0.398680,0.395503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,-0.021494,-0.024439,-0.021877,-0.018722,-0.018384,-0.019853,-0.020292,-0.018930,-0.017418,-0.017117,...,0.041869,0.043017,0.043672,0.044041,0.044656,0.045717,0.046863,0.047646,0.048089,0.048581
881,0.141533,0.164683,0.152672,0.136099,0.136327,0.146936,0.150647,0.144023,0.138333,0.141112,...,-0.457594,-0.457217,-0.456832,-0.456634,-0.456691,-0.456818,-0.456735,-0.456354,-0.455854,-0.455478
882,0.012251,0.018154,0.016136,0.017717,0.017060,0.017955,0.017464,0.018010,0.017709,0.018115,...,0.005854,0.006129,0.006400,0.006754,0.007096,0.007513,0.007718,0.008190,0.008366,0.008717
883,-0.066388,-0.097620,-0.086484,-0.094232,-0.089016,-0.093547,-0.090408,-0.093376,-0.091542,-0.093365,...,0.016241,0.015903,0.015626,0.015331,0.015115,0.014861,0.014656,0.014480,0.014234,0.013954


In [7]:
df_unhealthy.rename(columns={0 : 'label'}, inplace=True)
df_unhealthy['label'] = 1
df_unhealthy

,label,1,2,3,4,5,6,7,8,9,...,173215,173216,173217,173218,173219,173220,173221,173222,173223,173224
0,1,0.102660,0.091162,0.099346,0.093962,0.098952,0.095814,0.099173,0.097429,0.099446,...,0.032574,0.031380,0.030319,0.029251,0.028082,0.027002,0.025912,0.024975,0.023997,0.022895
1,1,-0.039935,-0.035308,-0.038430,-0.036246,-0.038055,-0.036828,-0.038055,-0.037291,-0.037937,...,0.014216,0.013896,0.013550,0.013139,0.012629,0.012183,0.011777,0.011292,0.010910,0.010378
2,1,-0.370539,-0.323283,-0.349609,-0.325975,-0.340078,-0.324729,-0.333104,-0.322754,-0.327030,...,-0.315960,-0.317101,-0.318015,-0.319889,-0.322268,-0.324422,-0.325804,-0.326805,-0.328887,-0.330444
3,1,0.000006,-0.000037,-0.000097,-0.000167,-0.000241,-0.000306,-0.000349,-0.000356,-0.000316,...,-0.057230,-0.058265,-0.059157,-0.059866,-0.060358,-0.060599,-0.060566,-0.060245,-0.059636,-0.058751
4,1,0.000900,0.000911,0.000785,0.000488,-0.000004,-0.000697,-0.001581,-0.002624,-0.003781,...,0.404312,0.405778,0.406606,0.406793,0.406343,0.405275,0.403615,0.401401,0.398680,0.395503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,1,-0.024439,-0.021877,-0.018722,-0.018384,-0.019853,-0.020292,-0.018930,-0.017418,-0.017117,...,0.041869,0.043017,0.043672,0.044041,0.044656,0.045717,0.046863,0.047646,0.048089,0.048581
881,1,0.164683,0.152672,0.136099,0.136327,0.146936,0.150647,0.144023,0.138333,0.141112,...,-0.457594,-0.457217,-0.456832,-0.456634,-0.456691,-0.456818,-0.456735,-0.456354,-0.455854,-0.455478
882,1,0.018154,0.016136,0.017717,0.017060,0.017955,0.017464,0.018010,0.017709,0.018115,...,0.005854,0.006129,0.006400,0.006754,0.007096,0.007513,0.007718,0.008190,0.008366,0.008717
883,1,-0.097620,-0.086484,-0.094232,-0.089016,-0.093547,-0.090408,-0.093376,-0.091542,-0.093365,...,0.016241,0.015903,0.015626,0.015331,0.015115,0.014861,0.014656,0.014480,0.014234,0.013954


In [8]:
df = pd.concat([df_healthy, df_unhealthy], axis=0)
df

,label,1,2,3,4,5,6,7,8,9,...,173215,173216,173217,173218,173219,173220,173221,173222,173223,173224
0,0,0.004234,0.003641,0.003971,0.003700,0.003809,0.003576,0.003688,0.003570,0.003566,...,0.008067,0.008140,0.008190,0.008175,0.008217,0.008314,0.008320,0.008422,0.008517,0.008680
1,0,-0.108564,-0.093635,-0.100122,-0.091679,-0.093961,-0.087420,-0.088095,-0.083677,-0.084414,...,0.001657,0.001447,0.000820,0.000003,-0.000793,-0.001621,-0.002303,-0.002842,-0.003067,-0.003418
2,0,0.045788,0.040219,0.043209,0.039028,0.038991,0.034904,0.033426,0.029972,0.028974,...,-0.014146,-0.012259,-0.009135,-0.005213,-0.000988,0.002983,0.006162,0.008906,0.010976,0.012153
3,0,0.043004,0.036623,0.038840,0.035038,0.035900,0.033086,0.033206,0.031031,0.030529,...,0.089253,0.088449,0.087667,0.086844,0.086066,0.085214,0.084552,0.083685,0.082866,0.082092
4,0,-0.084398,-0.071648,-0.076402,-0.070458,-0.074119,-0.071738,-0.075175,-0.074700,-0.077834,...,-0.206857,-0.213262,-0.218788,-0.223172,-0.226421,-0.228739,-0.230270,-0.231345,-0.232212,-0.233467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,1,-0.024439,-0.021877,-0.018722,-0.018384,-0.019853,-0.020292,-0.018930,-0.017418,-0.017117,...,0.041869,0.043017,0.043672,0.044041,0.044656,0.045717,0.046863,0.047646,0.048089,0.048581
881,1,0.164683,0.152672,0.136099,0.136327,0.146936,0.150647,0.144023,0.138333,0.141112,...,-0.457594,-0.457217,-0.456832,-0.456634,-0.456691,-0.456818,-0.456735,-0.456354,-0.455854,-0.455478
882,1,0.018154,0.016136,0.017717,0.017060,0.017955,0.017464,0.018010,0.017709,0.018115,...,0.005854,0.006129,0.006400,0.006754,0.007096,0.007513,0.007718,0.008190,0.008366,0.008717
883,1,-0.097620,-0.086484,-0.094232,-0.089016,-0.093547,-0.090408,-0.093376,-0.091542,-0.093365,...,0.016241,0.015903,0.015626,0.015331,0.015115,0.014861,0.014656,0.014480,0.014234,0.013954


In [9]:
# df.to_csv("D:/Data/호흡데이터/Task1/Wav_nonsegment/task1_nonsegment.csv")

In [10]:
# 라벨 열을 배열로 변환
label_arr = df['label'].values
label_arr

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
label_col = 'label' # 라벨이 되는 열 이름
data_cols = df.columns[df.columns != label_col].tolist() # 나머지 열 이름

In [12]:
label_col

'label'

In [13]:
X = df[data_cols]
y = df[label_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [15]:
# 라벨을 one-hot 인코딩으로 변환
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [16]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [17]:
def residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate=0.0):
    # 첫 번째 컨볼루션 레이어
    x_res = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(x)
    x_res = BatchNormalization()(x_res)
    x_res = Activation('relu')(x_res)

    # 두 번째 컨볼루션 레이어
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)

    # 세 번째 컨볼루션 레이어
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)

    # 스킵 연결 추가
    x = tf.keras.layers.add([x_res, x])

    return x


def TCN(nb_filters, kernel_size, dilations, padding, dropout_rate):
    # 입력 레이어
    inputs = Input(shape=(1,173224))

    # 임베딩 레이어
    x = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # residual blocks 쌓기
    for dilation_rate in dilations:
        x = residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate)

    # Global Average Pooling 레이어
    x = GlobalAveragePooling1D()(x)

    # 출력 레이어
    outputs = Dense(units=y_train.shape[1], activation='softmax')(x)

    # 모델 생성
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [18]:
nb_filters = 256  # 컨볼루션 필터 수
kernel_size = 2  # 컨볼루션 필터 크기
dilations = [2 ** i for i in range(8)] * 2  # dilation rate
padding = 'same'
dropout_rate = 0.4  # dropout 비율

# 모델 생성
model = TCN(nb_filters, kernel_size, dilations, padding, dropout_rate)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()
# 모델 훈련
# history = model.fit(X_train, y_train, batch_size=128, epochs=100)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 173224)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 1, 256)       44345600    ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 1, 256)      1024        ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 1, 256)       0           ['batch_normalization[0][0]']

In [109]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

6/6 [==============================] - 1s 63ms/step - loss: 0.2793 - accuracy: 0.9511
Test loss: 0.2793
Test accuracy: 0.9511


In [100]:
# train set에서 최고 정확도 출력
train_acc = max(history.history['accuracy'])
print(f"best train accuracy: {train_acc}")

# validation set에서 최고 정확도 출력
val_acc = max(history.history['val_accuracy'])
print(f"best validation accuracy: {val_acc}")

best train accuracy: 0.95923912525177
best validation accuracy: 0.9728260636329651


# Task1_Segment

In [69]:
## 자르기

# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_segment/Healthy'
# 읽어올 폴더 경로 설정(데이터 길이 구하기 위함)
data_folder_unhealthy = 'D:/Data/호흡데이터/Task1/Wav_segment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list = []
for root, dirs, files in os.walk(data_folder):
    for file in files:
        if file.endswith('.wav'):
            file_list.append(os.path.join(root, file))
            
            
# 데이터 길이 맞추기위해 읽어올 파일 리스트 생성
file_list_unhealthy = []
for root, dirs, files_unhealthy in os.walk(data_folder_unhealthy):
    for file_unhealthy in files_unhealthy:
        if file.endswith('.wav'):
            file_list_unhealthy.append(os.path.join(root, file_unhealthy))
            

# 가장 짧은 파일의 길이 구하기
min_len = min([len(librosa.load(file_unhealthy)[0]) for file_unhealthy in file_list_unhealthy])

# 데이터프레임 생성
dfs = []
for file in file_list:
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 가장 짧은 길이에 맞게 데이터 자르기
    audio_data = audio_data[:min_len]
    # 시간 축 설정
    time_axis = librosa.times_like(audio_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(audio_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_healthy_segment = pd.concat(dfs, axis=1, ignore_index=True)

1it [00:00, 1003.42it/s]


KeyboardInterrupt: 

In [75]:
## 늘리기

# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_segment/Healthy'

Unhealthy_folder = 'D:/Data/호흡데이터/Task1/Wav_segment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list_unhealthy = []
for root, dirs, files in tqdm(os.walk(Unhealthy_folder)):
    for file in files:
        if file.endswith('.wav'):
            file_list_unhealthy.append(os.path.join(root, file))

# 가장 긴 파일의 길이 구하기
max_len = max([len(librosa.load(file)[0]) for file in file_list_unhealthy])

# 데이터프레임 생성
dfs = []
for file in tqdm(file_list):
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 평균값 구하기
    mean_value = np.mean(audio_data)
    # 패딩할 길이 계산하기
    padding_len = max_len - len(audio_data)
    # 패딩 추가하기
    padded_data = np.pad(audio_data, (0, padding_len), mode='constant', constant_values=mean_value)
    # 시간 축 설정
    time_axis = librosa.times_like(padded_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(padded_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_healthy_segment = pd.concat(dfs, axis=1, ignore_index=True)

1it [00:00, 47.73it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 322/322 [00:16<00:00, 20.04it/s]


In [76]:
df_healthy_segment = df_healthy_segment.transpose()
df_healthy_segment

,0,1,2,3,4,5,6,7,8,9,...,361258,361259,361260,361261,361262,361263,361264,361265,361266,361267
0,0.031820,0.046845,0.041362,0.045014,0.042429,0.044503,0.042943,0.044295,0.043381,0.044117,...,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510
1,-0.010591,-0.015574,-0.013833,-0.014993,-0.014043,-0.014767,-0.014348,-0.014734,-0.014374,-0.014731,...,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185
2,0.006721,0.009853,0.008797,0.009538,0.008882,0.009197,0.008704,0.008865,0.008397,0.008483,...,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691
3,-0.014312,-0.021034,-0.018488,-0.019934,-0.018670,-0.019521,-0.018758,-0.019308,-0.018987,-0.019389,...,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300
4,-0.035999,-0.052770,-0.046424,-0.050493,-0.047534,-0.049899,-0.048158,-0.049694,-0.048599,-0.049473,...,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.029142,0.042524,0.037193,0.040299,0.037739,0.039439,0.037841,0.038833,0.037786,0.038285,...,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167
318,0.047693,0.069728,0.061119,0.066179,0.062124,0.065085,0.062366,0.064121,0.062450,0.063499,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
319,-0.044659,-0.065428,-0.057655,-0.062668,-0.059073,-0.061892,-0.059588,-0.061378,-0.059846,-0.060954,...,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444
320,0.041090,0.060298,0.053246,0.057850,0.054521,0.057193,0.055197,0.056892,0.055551,0.056429,...,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686


In [78]:
df_healthy_segment.rename(columns={0 : 'label'}, inplace=True)
df_healthy_segment['label'] = 0
df_healthy_segment

,label,1,2,3,4,5,6,7,8,9,...,361258,361259,361260,361261,361262,361263,361264,361265,361266,361267
0,0,0.046845,0.041362,0.045014,0.042429,0.044503,0.042943,0.044295,0.043381,0.044117,...,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510
1,0,-0.015574,-0.013833,-0.014993,-0.014043,-0.014767,-0.014348,-0.014734,-0.014374,-0.014731,...,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185
2,0,0.009853,0.008797,0.009538,0.008882,0.009197,0.008704,0.008865,0.008397,0.008483,...,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691
3,0,-0.021034,-0.018488,-0.019934,-0.018670,-0.019521,-0.018758,-0.019308,-0.018987,-0.019389,...,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300
4,0,-0.052770,-0.046424,-0.050493,-0.047534,-0.049899,-0.048158,-0.049694,-0.048599,-0.049473,...,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0,0.042524,0.037193,0.040299,0.037739,0.039439,0.037841,0.038833,0.037786,0.038285,...,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167
318,0,0.069728,0.061119,0.066179,0.062124,0.065085,0.062366,0.064121,0.062450,0.063499,...,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426,-0.000426
319,0,-0.065428,-0.057655,-0.062668,-0.059073,-0.061892,-0.059588,-0.061378,-0.059846,-0.060954,...,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444,0.001444
320,0,0.060298,0.053246,0.057850,0.054521,0.057193,0.055197,0.056892,0.055551,0.056429,...,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686,-0.002686


In [70]:
### 자르기

# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_segment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list = []
for root, dirs, files in tqdm(os.walk(data_folder)):
    for file in files:
        if file.endswith('.wav'):
            file_list.append(os.path.join(root, file))

# 가장 짧은 파일의 길이 구하기
min_len = min([len(librosa.load(file)[0]) for file in file_list])

# 데이터프레임 생성
dfs = []
for file in tqdm(file_list):
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 가장 짧은 길이에 맞게 데이터 자르기
    audio_data = audio_data[:min_len]
    # 시간 축 설정
    time_axis = librosa.times_like(audio_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(audio_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_unhealthy_segment = pd.concat(dfs, axis=1, ignore_index=True)

1it [00:00, 46.77it/s]


KeyboardInterrupt: 

In [71]:
## 늘리기

# 읽어올 폴더 경로 설정
data_folder = 'D:/Data/호흡데이터/Task1/Wav_segment/Unhealthy'

# 읽어올 파일 리스트 생성
file_list = []
for root, dirs, files in tqdm(os.walk(data_folder)):
    for file in files:
        if file.endswith('.wav'):
            file_list.append(os.path.join(root, file))

# 가장 긴 파일의 길이 구하기
max_len = max([len(librosa.load(file)[0]) for file in file_list])

# 데이터프레임 생성
dfs = []
for file in tqdm(file_list):
    # 오디오 파일 읽어오기
    audio_data, sample_rate = librosa.load(file)
    # 평균값 구하기
    mean_value = np.mean(audio_data)
    # 패딩할 길이 계산하기
    padding_len = max_len - len(audio_data)
    # 패딩 추가하기
    padded_data = np.pad(audio_data, (0, padding_len), mode='constant', constant_values=mean_value)
    # 시간 축 설정
    time_axis = librosa.times_like(padded_data, sr=sample_rate)
    # 채널 축 설정
    channel_axis = ['Channel 1']
    # 데이터프레임 생성
    df = pd.DataFrame(padded_data.reshape(-1, len(channel_axis)), columns=channel_axis)
    
    dfs.append(df)

# 모든 데이터프레임을 하나로 합치기
df_unhealthy_segment = pd.concat(dfs, axis=1, ignore_index=True)

1it [00:00, 46.43it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 6576/6576 [06:23<00:00, 17.15it/s]


In [72]:
df_unhealthy_segment = df_unhealthy_segment.transpose()
df_unhealthy_segment

,0,1,2,3,4,5,6,7,8,9,...,361258,361259,361260,361261,361262,361263,361264,361265,361266,361267
0,-0.002457,-0.003693,-0.003417,-0.003740,-0.003609,-0.003878,-0.003735,-0.003992,-0.003988,-0.004031,...,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869
1,-0.068233,-0.099915,-0.087809,-0.095247,-0.089586,-0.093977,-0.090251,-0.092868,-0.090588,-0.092129,...,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363
2,0.019185,0.028217,0.024996,0.027193,0.025651,0.027094,0.026221,0.026985,0.026474,0.027070,...,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498
3,0.018328,0.026931,0.023891,0.026022,0.024644,0.025983,0.025059,0.025753,0.025256,0.025655,...,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656
4,0.040589,0.059426,0.052245,0.056828,0.053359,0.055949,0.053820,0.055476,0.054281,0.055336,...,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,0.122221,0.179154,0.157903,0.171524,0.161279,0.168878,0.162411,0.167137,0.163014,0.165699,...,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301
6572,0.004716,0.006466,0.005275,0.005454,0.004694,0.004632,0.004100,0.003986,0.003438,0.003049,...,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275
6573,0.024501,0.035796,0.031475,0.034012,0.031747,0.033035,0.031451,0.032101,0.031015,0.031319,...,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198
6574,-0.034041,-0.049789,-0.043592,-0.047203,-0.044249,-0.046084,-0.044158,-0.045249,-0.043926,-0.044602,...,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203


In [77]:
df_unhealthy_segment.rename(columns={0 : 'label'}, inplace=True)
df_unhealthy_segment['label'] = 1
df_unhealthy_segment

,label,1,2,3,4,5,6,7,8,9,...,361258,361259,361260,361261,361262,361263,361264,361265,361266,361267
0,1,-0.003693,-0.003417,-0.003740,-0.003609,-0.003878,-0.003735,-0.003992,-0.003988,-0.004031,...,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869,-0.003869
1,1,-0.099915,-0.087809,-0.095247,-0.089586,-0.093977,-0.090251,-0.092868,-0.090588,-0.092129,...,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363,-0.000363
2,1,0.028217,0.024996,0.027193,0.025651,0.027094,0.026221,0.026985,0.026474,0.027070,...,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498,0.000498
3,1,0.026931,0.023891,0.026022,0.024644,0.025983,0.025059,0.025753,0.025256,0.025655,...,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656,0.000656
4,1,0.059426,0.052245,0.056828,0.053359,0.055949,0.053820,0.055476,0.054281,0.055336,...,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444,-0.000444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,1,0.179154,0.157903,0.171524,0.161279,0.168878,0.162411,0.167137,0.163014,0.165699,...,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301
6572,1,0.006466,0.005275,0.005454,0.004694,0.004632,0.004100,0.003986,0.003438,0.003049,...,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275
6573,1,0.035796,0.031475,0.034012,0.031747,0.033035,0.031451,0.032101,0.031015,0.031319,...,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198
6574,1,-0.049789,-0.043592,-0.047203,-0.044249,-0.046084,-0.044158,-0.045249,-0.043926,-0.044602,...,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203


In [79]:
df = pd.concat([df_healthy_segment, df_unhealthy_segment], axis=0)
df

,label,1,2,3,4,5,6,7,8,9,...,361258,361259,361260,361261,361262,361263,361264,361265,361266,361267
0,0,0.046845,0.041362,0.045014,0.042429,0.044503,0.042943,0.044295,0.043381,0.044117,...,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510,0.000510
1,0,-0.015574,-0.013833,-0.014993,-0.014043,-0.014767,-0.014348,-0.014734,-0.014374,-0.014731,...,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185,0.000185
2,0,0.009853,0.008797,0.009538,0.008882,0.009197,0.008704,0.008865,0.008397,0.008483,...,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691,0.000691
3,0,-0.021034,-0.018488,-0.019934,-0.018670,-0.019521,-0.018758,-0.019308,-0.018987,-0.019389,...,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300,-0.001300
4,0,-0.052770,-0.046424,-0.050493,-0.047534,-0.049899,-0.048158,-0.049694,-0.048599,-0.049473,...,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582,0.000582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,1,0.179154,0.157903,0.171524,0.161279,0.168878,0.162411,0.167137,0.163014,0.165699,...,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301,0.002301
6572,1,0.006466,0.005275,0.005454,0.004694,0.004632,0.004100,0.003986,0.003438,0.003049,...,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275,0.000275
6573,1,0.035796,0.031475,0.034012,0.031747,0.033035,0.031451,0.032101,0.031015,0.031319,...,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198,-0.000198
6574,1,-0.049789,-0.043592,-0.047203,-0.044249,-0.046084,-0.044158,-0.045249,-0.043926,-0.044602,...,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203,-0.000203


In [80]:
label_col = 'label' # 라벨이 되는 열 이름
data_cols = df.columns[df.columns != label_col].tolist() # 나머지 열 이름

X = df[data_cols]
y = df[label_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [81]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [63]:
y_test

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [82]:
# 라벨을 one-hot 인코딩으로 변환
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [28]:
def residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate=0.0):
    # 첫 번째 컨볼루션 레이어
    x_res = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(x)
    x_res = BatchNormalization()(x_res)
    x_res = Activation('relu')(x_res)

    # 두 번째 컨볼루션 레이어
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)

    # 세 번째 컨볼루션 레이어
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size,
               dilation_rate=dilation_rate, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(dropout_rate)(x)
    
    # 스킵 연결 추가
    x = tf.keras.layers.add([x_res, x])

    return x


def TCN(nb_filters, kernel_size, dilations, padding, dropout_rate):
    # 입력 레이어
    inputs = Input(shape=(1,361267))

    # 임베딩 레이어
    x = Conv1D(filters=nb_filters, kernel_size=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # residual blocks 쌓기
    for dilation_rate in dilations:
        x = residual_block(x, dilation_rate, nb_filters, kernel_size, padding, dropout_rate)

    # Global Average Pooling 레이어
    x = GlobalAveragePooling1D()(x)

    # 출력 레이어
    outputs = Dense(units=y_train.shape[1], activation='softmax')(x)

    # 모델 생성
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [29]:
nb_filters = 128  # 컨볼루션 필터 수
kernel_size = 3  # 컨볼루션 필터 크기
dilations = [2 ** i for i in range(8)] * 2  # dilation rate
padding = 'same'
dropout_rate = 0.3  # dropout 비율

# 모델 생성
model = TCN(nb_filters, kernel_size, dilations, padding, dropout_rate)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 모델 훈련
# history = model.fit(X_train, y_train, batch_size=128, epochs=50)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1, 361267)]  0           []                               
                                                                                                  
 conv1d_198 (Conv1D)            (None, 1, 128)       46242304    ['input_7[0][0]']                
                                                                                                  
 batch_normalization_198 (Batch  (None, 1, 128)      512         ['conv1d_198[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_198 (Activation)    (None, 1, 128)       0           ['batch_normalization_198[0

In [90]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

44/44 [==============================] - 3s 53ms/step - loss: 0.1949 - accuracy: 0.9486
Test loss: 0.1949
Test accuracy: 0.9486
